In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from train import create_challenging_agent_dist
from utils import (
    compute_continuity_noise,
    fixed_point_interpolation_true_distribution,
    convert_to_polar_coordinates,
)
from optimal_beta import expected_policy_loss

In [2]:
df = pd.read_csv("results/low_dim_11-25-22.csv")
df
#df.groupby("method").agg(["mean", "std"])

,n,d,n_types,sigma,q,seed,opt_loss,opt_theta,final_loss,final_theta,method
0,1000000,2,10,3.411968,0.7,0,-1.614850,1.154054,-1.479698,0.000000,ewm
1,1000000,2,10,3.275181,0.7,4,-1.703331,1.282283,-1.515607,0.000000,ewm
2,1000000,2,10,3.396843,0.7,5,-1.548821,1.154054,-1.403792,6.283185,ewm
3,1000000,2,10,3.478472,0.7,3,-1.560043,1.282283,-1.372854,6.283185,ewm
4,1000000,2,10,3.482621,0.7,1,-1.604159,1.154054,-1.361325,6.283185,ewm
5,1000000,2,10,2.951452,0.7,6,-1.615830,1.154054,-1.420685,6.283185,ewm
6,1000000,2,10,3.316494,0.7,2,-1.495171,1.282283,-1.301076,6.283185,ewm
7,1000000,2,10,3.419998,0.7,7,-1.647061,1.154054,-1.469453,0.000000,ewm
8,1000000,2,10,2.850814,0.7,8,-1.544511,1.410511,-1.369299,6.283185,ewm
9,1000000,2,10,3.463803,0.7,9,-1.717866,1.154054,-1.471832,6.283185,ewm


In [12]:
methods = ["ewm", "total_deriv", "partial_deriv_loss_theta"]

for method in methods:
    subdf = df[df.method == method]
    print(method)
    z = np.abs(subdf["opt_loss"] - subdf["final_loss"]).mean()
    z2 = (subdf["opt_loss"] - subdf["final_loss"]).std()
    print("Loss Diff: {} +- {}".format(round(z, 3), round(z2, 3)))
    
    w = np.abs(subdf["opt_theta"] - subdf["final_theta"]).mean()
    w2 = (subdf["opt_theta"] - subdf["final_theta"]).std()
    
    print("Theta Diff: {} +- {}".format(round(w, 3), round(w2, 3)))
    print("------")

ewm
Loss Diff: 0.189 +- 0.036
Theta Diff: 3.898 +- 3.022
------
total_deriv
Loss Diff: 0.001 +- 0.0
Theta Diff: 0.043 +- 0.051
------
partial_deriv_loss_theta
Loss Diff: 0.037 +- 0.047
Theta Diff: 0.325 +- 0.316
------


In [13]:
total_deriv_df = df[df.method=="total_deriv"]
(total_deriv_df["opt_loss"] - total_deriv_df["final_loss"]).mean(), np.abs(total_deriv_df["opt_loss"] - total_deriv_df["final_loss"]).std()

(-0.0006977979304476411, 0.00031632739103074184)

In [ ]:
np.abs(total_deriv_df["opt_theta"] - total__df["final_theta"]).mean(), np.abs(ewm_df["opt_theta"] - ewm_df["final_theta"]).std()

In [17]:
SEEDS = list(range(10))
naive_losses = []
n=1000000
n_types=10
d=2
beta = np.array([1., 0.]).reshape(2, 1)
true_beta = beta.copy()
thetas = []

for seed in SEEDS:
    np.random.seed(seed)
    
    agent_dist = create_challenging_agent_dist(n, n_types, d)
    sigma = compute_continuity_noise(agent_dist) + 0.05
    q = 0.7
    f = fixed_point_interpolation_true_distribution(
            agent_dist, sigma, q, plot=False, savefig=None
        )

    s = f(convert_to_polar_coordinates(beta).item())
    thetas.append(convert_to_polar_coordinates(beta).item())
    
    naive_losses.append(expected_policy_loss(agent_dist, beta, s, sigma, true_beta=true_beta))

100%|█████████████████████████████████████████| 100/100 [00:33<00:00,  3.00it/s]


In [18]:
naive_losses = np.array(naive_losses)
thetas = np.array(thetas)
(naive_losses - df["opt_loss"]).mean(), (naive_losses - df["opt_loss"]).std()

(0.5907676914401055, 0.19693053407705016)

In [23]:
np.abs(thetas - df["opt_theta"]).mean(), np.abs(thetas - df["opt_theta"]).std()

NameError: name 'thetas' is not defined